In [1]:
import plotly.express as px
import pandas as pd
from config import StocksConfig as config
import psycopg2

conn = psycopg2.connect(database=config.DB_NAME, host=config.DB_HOST, user=config.DB_USER, password=config.DB_PASS, port=config.DB_PORT)

In [2]:
query = """
SELECT symbol, time_bucket('1 day', stock_datetime) AS time_frame, last(price_close, stock_datetime) AS last_closing_price
FROM stocks_new
WHERE symbol = 'AAPL'
GROUP BY time_frame, symbol
ORDER BY time_frame
"""

df = pd.read_sql(query, conn)

figure = px.line(df, x='time_frame', y='last_closing_price')
figure.show()

In [6]:
query = """
SELECT symbol, time_bucket('7 days', stock_datetime) AS time_frame, sum(trading_volume) AS volume
FROM stocks_new
WHERE symbol = 'IBM'
GROUP BY time_frame, symbol
ORDER BY time_frame
"""

df = pd.read_sql(query, conn)

figure = px.line(df, x='time_frame', y='volume')
figure.show()

In [11]:
query = """
SELECT t.symbol, t.name, sum(trading_volume) AS volume
FROM stocks_new
INNER JOIN ticker t ON t.symbol = stocks_new.symbol
WHERE (now() - date(stock_datetime)) < INTERVAL '3 MONTH'
GROUP BY t.symbol, t.name
ORDER BY volume DESC
LIMIT 5
"""

df = pd.read_sql(query, conn)

figure = px.bar(df, x='name', y='volume')
figure.show()

In [17]:
query = """
SELECT symbol, time_bucket('10 min', stock_datetime) AS time_frame, last(price_close, stock_datetime) AS last_closing_price
FROM stocks_new
WHERE symbol IN ('AAPL', 'FB', 'GOOG', 'AMZN', 'NFLX')
GROUP BY time_frame, symbol
ORDER BY time_frame
"""

df = pd.read_sql(query, conn)

figure = px.line(df, x='time_frame', y='last_closing_price', color='symbol')
figure.show()